# Feature Engineering

In [23]:
import pandas as pd
import numpy as np
import sqlite3
pd.set_option("display.max_columns", None)

conn = sqlite3.connect("../data/ufc.db", detect_types=sqlite3.PARSE_DECLTYPES)
query = """
        SELECT
            a.*,
            b.HEIGHT_INCHES AS RED_HEIGHT_INCHES,
            b.REACH_INCHES AS RED_REACH_INCHES,
            b.FIGHTING_STANCE AS RED_FIGHTING_STANCE,
            b.DATE_OF_BIRTH AS RED_DATE_OF_BIRTH,
            c.HEIGHT_INCHES AS BLUE_HEIGHT_INCHES,
            c.REACH_INCHES AS BLUE_REACH_INCHES,
            c.FIGHTING_STANCE AS BLUE_FIGHTING_STANCE,
            c.DATE_OF_BIRTH AS BLUE_DATE_OF_BIRTH
        FROM
            BOUTS_OVERALL a
        LEFT JOIN
            FIGHTERS b
        ON
            a.RED_FIGHTER_ID = b.FIGHTER_ID
        LEFT JOIN
            FIGHTERS c
        ON
            a.BLUE_FIGHTER_ID = c.FIGHTER_ID
        """

df = pd.read_sql_query(query, conn)

## Create dataframes to be used to generate features

In [25]:
temp = df.copy().rename_axis("TEMP").reset_index()
common = ["TEMP", "BOUT_ID", "DATE", "LOCATION", "BOUT_TYPE", "OUTCOME_METHOD", "END_ROUND", "BOUT_TIME_FORMAT", "TOTAL_TIME_MINUTES"]
red_cols = [col for col in temp.columns if col.startswith("RED_")]
blue_cols = [col for col in temp.columns if col.startswith("BLUE_")]

reds = temp[common + red_cols + blue_cols]
reds = reds.rename(columns=lambda x: x.replace("RED_", "f_") if x.startswith("RED_") else x.replace("BLUE_", "o_") if x.startswith("BLUE_") else x)
reds["TEMP2"] = 1
blues = temp[common + blue_cols + red_cols]
blues = blues.rename(columns=lambda x: x.replace("BLUE_", "f_") if x.startswith("BLUE_") else x.replace("RED_", "o_") if x.startswith("RED_") else x)
blues["TEMP2"] = 2

feature_df = pd.concat([reds, blues]).sort_values(["TEMP", "TEMP2"]).drop(["TEMP", "TEMP2", "f_FIGHTER_NAME", "o_FIGHTER_NAME"], axis=1)
feature_df["DATE"] = pd.to_datetime(feature_df["DATE"])
feature_df["f_DATE_OF_BIRTH"] = pd.to_datetime(feature_df["f_DATE_OF_BIRTH"])
feature_df["o_DATE_OF_BIRTH"] = pd.to_datetime(feature_df["o_DATE_OF_BIRTH"])
feature_df

,BOUT_ID,DATE,LOCATION,BOUT_TYPE,OUTCOME_METHOD,END_ROUND,BOUT_TIME_FORMAT,TOTAL_TIME_MINUTES,f_FIGHTER_ID,f_OUTCOME,f_KNOCKDOWNS,f_TOTAL_STRIKES_LANDED,f_TOTAL_STRIKES_ATTEMPTED,f_TAKEDOWNS_LANDED,f_TAKEDOWNS_ATTEMPTED,f_SUBMISSION_ATTEMPTS,f_REVERSALS,f_CONTROL_TIME_MINUTES,f_SIGNIFICANT_STRIKES_LANDED,f_SIGNIFICANT_STRIKES_ATTEMPTED,f_SIGNIFICANT_STRIKES_HEAD_LANDED,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,f_SIGNIFICANT_STRIKES_BODY_LANDED,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,f_SIGNIFICANT_STRIKES_LEG_LANDED,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,f_SIGNIFICANT_STRIKES_CLINCH_LANDED,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,f_SIGNIFICANT_STRIKES_GROUND_LANDED,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,f_HEIGHT_INCHES,f_REACH_INCHES,f_FIGHTING_STANCE,f_DATE_OF_BIRTH,o_FIGHTER_ID,o_OUTCOME,o_KNOCKDOWNS,o_TOTAL_STRIKES_LANDED,o_TOTAL_STRIKES_ATTEMPTED,o_TAKEDOWNS_LANDED,o_TAKEDOWNS_ATTEMPTED,o_SUBMISSION_ATTEMPTS,o_REVERSALS,o_CONTROL_TIME_MINUTES,o_SIGNIFICANT_STRIKES_LANDED,o_SIGNIFICANT_STRIKES_ATTEMPTED,o_SIGNIFICANT_STRIKES_HEAD_LANDED,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,o_SIGNIFICANT_STRIKES_BODY_LANDED,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,o_SIGNIFICANT_STRIKES_LEG_LANDED,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,o_SIGNIFICANT_STRIKES_CLINCH_LANDED,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,o_SIGNIFICANT_STRIKES_GROUND_LANDED,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,o_HEIGHT_INCHES,o_REACH_INCHES,o_FIGHTING_STANCE,o_DATE_OF_BIRTH
0,567a09fd200cfa05,1993-11-12,"Denver, Colorado, USA",Open Weight Bout,KO/TKO,1,No Time Limit,0.433333,279093302a6f44b3,W,0.0,3.0,5.0,0.0,0.0,0.0,0.0,NaN,3.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,2.0,77.0,NaN,Orthodox,1959-03-30,96eff1a628adcc7f,L,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,72.0,NaN,Orthodox,1969-06-14
0,567a09fd200cfa05,1993-11-12,"Denver, Colorado, USA",Open Weight Bout,KO/TKO,1,No Time Limit,0.433333,96eff1a628adcc7f,L,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,72.0,NaN,Orthodox,1969-06-14,279093302a6f44b3,W,0.0,3.0,5.0,0.0,0.0,0.0,0.0,NaN,3.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,2.0,77.0,NaN,Orthodox,1959-03-30
1,2d2bbc86e941e05c,1993-11-12,"Denver, Colorado, USA",Open Weight Bout,KO/TKO,1,No Time Limit,4.333333,598a58db87b890ee,W,2.0,38.0,53.0,0.0,0.0,0.0,0.0,NaN,15.0,27.0,12.0,23.0,3.0,4.0,0.0,0.0,4.0,10.0,4.0,9.0,7.0,8.0,76.0,NaN,Orthodox,NaT,d3711d3784b76255,L,0.0,13.0,29.0,0.0,0.0,0.0,0.0,NaN,12.0,28.0,7.0,19.0,3.0,6.0,2.0,3.0,0.0,7.0,10.0,19.0,2.0,2.0,77.0,NaN,Orthodox,NaT
1,2d2bbc86e941e05c,1993-11-12,"Denver, Colorado, USA",Open Weight Bout,KO/TKO,1,No Time Limit,4.333333,d3711d3784b76255,L,0.0,13.0,29.0,0.0,0.0,0.0,0.0,NaN,12.0,28.0,7.0,19.0,3.0,6.0,2.0,3.0,0.0,7.0,10.0,19.0,2.0,2.0,77.0,NaN,Orthodox,NaT,598a58db87b890ee,W,2.0,38.0,53.0,0.0,0.0,0.0,0.0,NaN,15.0,27.0,12.0,23.0,3.0,4.0,0.0,0.0,4.0,10.0,4.0,9.0,7.0,8.0,76.0,NaN,Orthodox,NaT
2,cecdc0da584274b9,1993-11-12,"Denver, Colorado, USA",Open Weight Bout,Submission,1,No Time Limit,2.300000,429e7d3725852ce9,W,0.0,4.0,7.0,1.0,1.0,0.0,0.0,NaN,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,73.0,NaN,Southpaw,1966-12-12,a5c53b3ddb31cc7d,L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,NaN,Orthodox,1963-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7261,eff6ec2b2f1fb27b,2023-07-29,"Salt Lake City, Utah, USA",Heavyweight Bout,KO/TKO,1,3 Rnd (5-5-5),0.550000,ab943cd2c3c17825,L,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,73.0,75.0,Orthodox,1985-06-25,d3df1add9d9a7efb,